In [1]:
import sys


In [3]:
sys.argv.pop()

'-f'

In [4]:
sys.argv

['/Users/aaronciuffo/Virtualenvs/python2/lib/python2.7/site-packages/ipykernel/__main__.py']

In [5]:
import sys
import re
import datetime
import unicodedata

# I kind of hate this
#from __future__ import print_function

# this may be unecessary - double check
import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage 
from apiclient import errors as gerror

#SCOPES = 'https://www.googleapis.com/auth/drive.file'
SCOPES = 'https://www.googleapis.com/auth/drive.file' # try this one!
# SCOPES = 'https://www.googleapis.com/auth/drive.metadata.readonly'
CLIENT_SECRET_FILE = 'client_secret_qs.json'
APPLICATION_NAME = 'Drive API Python Quickstart'

In [6]:
def strip_accents(s):
    s = unicode(s, "utf-8")
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


In [7]:
def get_valid_filename(s):
    """
    Return the given string converted to a string that can be used for a clean
    filename. Remove leading and trailing spaces
    """
    s = s.strip()
    s = strip_accents(s)
    return re.sub(r'(?u)[^-\w., ]', '', s)

In [8]:
def fileRead(fname):
    '''
    read a file into a list, strip out all accented and special characters, leading spaces
    '''
    lines = []
    try:
        with open(fname) as f:
            for each in f:
                each = get_valid_filename(each)
                lines.append(each.strip('\n'))
            return(lines)
    except (OSError, IOError) as e:
        print 'error reading file:', fname, e
        return(False)

In [9]:
def pathify(parts = [], basepath = ''):
    '''
    create a path from a list of strings
    
    accepts:
        * parts (list): list of strings ['part_one', 'part_two', 'part_three']
        * basepath (string): string to append to the start of the path (default: ./)
    
    returns:
        * path: basePath/part_one/part_two/part_three/
    '''
    path = basepath
    if len(parts ) > 0:
        for each in parts:
            if re.match('.*\/$', each):
                path = path + each
            else:
                path = path + each + '/'
    return(path)

In [10]:
def getCredentials():
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        try:
            os.makedirs(credential_dir)
        except (IOError, OSError) as e:
            print(e)
        

    credential_path = os.path.join(credential_dir, 'drive-python.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = 'foo'
        credentials = tools.run_flow(flow, store, None)
    return(credentials)

In [11]:
class gDrive():
    '''
    creates a google drive interface object
    
    Accepts:
    google drive v3 service object: (discover.build('drive', 'v3', credentials = credentials_object)
    
    '''
    def __init__(self, object):
        self.service = object
        # https://developers.google.com/drive/v3/web/mime-types
        self.mimeTypes = {'audio': 'application/vnd.google-apps.audio',
                          'docs': 'application/vnd.google-apps.document',
                          'drawing': 'application/vnd.google-apps.drawing',
                          'file': 'application/vnd.google-apps.file',
                          'folder': 'application/vnd.google-apps.folder',
                          'forms': 'application/vnd.google-apps.form',
                          'mymaps': 'application/vnd.google-apps.map',
                          'photos': 'application/vnd.google-apps.photo',
                          'slides': 'application/vnd.google-apps.presentation',
                          'scripts': 'application/vnd.google-apps.script',
                          'sites': 'application/vnd.google-apps.sites',
                          'sheets': 'application/vnd.google-apps.spreadsheet',
                          'video': 'application/vnd.google-apps.video'}
    
#     types = property()
    
    @property
    def types(self):
        print('supported mime types:')
        for key in self.mimeTypes:
            #print('%10s: %s' % (key, self.mimeTypes[key]))
            print('{:8} {val}'.format(key+':', val=self.mimeTypes[key]))
    
    def quote(self, string):
        '''
        add double quotes arounda string
        '''
        return('"'+str(string)+'"')
    

    
    def addFile(self, name = None, ftype = False, parents = None):
        body={}
        if name is None:
            print('specify a folder or file name')
            return(False)
        else:
            body['name'] = name
        
        if ftype in self.mimeTypes:
            body['mimeType'] = self.mimeTypes[ftype]
        
        if isinstance(parents, list):
            body['parents'] = parents
        elif parents:
            body['parents'] = [parents]
        
        print(body)
        result = self.service.files().create(body=body).execute()
        
        #body = {'name':'release the schmoo!', 'mimeType':'application/vnd.google-apps.folder', 'parents':["0BzC-V2QIsGRGWXNxNmhjc0FITDQ"]}
# service.files().create(body=body).execute()
        return(result)

    
    def search(self, name = None, trashed = None, ftype = False, fuzzy = False, date = None, dopperator = '>', 
               parent = None):
        '''
        search for an item by name and other properties in google drive
        
        accepts:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            ftype = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parent = (string): google drive file id string
            
        returns:
            list of file dict
        '''
        features = ['name', 'trashed', 'ftype', 'date', 'parent']
        build = {'name' : 'name {} "{}"'.format(('contains' if fuzzy else '='), name),
                 'trashed' : 'trashed={}'. format(trashed),
                 'ftype' : 'mimeType="{}"'.format(self.mimeTypes[ftype] if ftype in self.mimeTypes else ''),
                 'parent': '"{}" in parents'.format(parent),
                 'date': 'modifiedTime{}"{}"'.format(dopperator, date)}

        # provides for setting trashed to True/False if the input is not None
        if not isinstance(trashed, type(None)):
            # set to true as the variable is now in use, but it's value has been set above
            trashed = True

        qList = []

        # evaluate feature options; if they are != None/False, use them in building query
        for each in features:
            if eval(each):
                qList.append(build[each])
                
        print(' and '.join(qList))
                
        try:
            # build a query with "and" statements
            result = self.service.files().list(q=' and '.join(qList)).execute()
            return(result)
        except gerror.HttpError as e:
            print('File Not Found')
            return(False)

    def ls(self, *args, **kwargs):
        result = self.search(*args, **kwargs)
        for eachFile in result.get('files', []):
            print('name: {f[name]}, ID:{f[id]}, mimeType:{f[mimeType]}'.format(f=eachFile))
            
    
    def parents(self, fid):
        """Print a file's parents.

        Args:
        service: Drive API service instance.
        file_id: ID of the file to print parents for.
        """
# service.files().get(fileId="0BzC-V2QIsGRGT00xSjJHNy1Td1k", fields='parents').execute()
        try:
            parents = self.service.files().get(fileId=fid, fields='parents').execute()
            return(parents)
        except gerror.HttpError as e:
            print ('An error occurred: {}'.format(e))
            return(False)
    
    def rm(self):
        pass
    
#     def search(self, name = None, trashed = False, ftype = False, fuzzy = False, date = None, parent = None):
#         '''
#         search for an item by name and other properties in google drive
        
#         accepts:
#             name (string): item name in google drive - required
#             trashed (bool): item is not in trash - default False
#             ftype = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
#             fuzzy = (bool): substring search of names in drive
#             date = (RFC3339 date string): currently searches for files created after date string (YYYY-MM-DD)
#             parent = (string): google drive file id string
            
#         returns:
#             list of file dict
#         '''
#         queryDict={}
#         if name is None:
#             print('specify a folder or file name')
#             return(False)
#         else:
#             if fuzzy:
#                 fVal = ' contains '
#             else:
#                 fVal = '='
#             queryDict['name'] = 'name'+fVal+self.quote(name)
            
#         if isinstance(trashed, bool):
#             if trashed:
#                 tVal='trashed=True'
#             else:
#                 tVal='trashed=False'
#             queryDict['trashed'] = tVal
        
#         if ftype in self.mimeTypes:
#             queryDict['mimeType'] = 'mimeType='+self.quote(self.mimeTypes[ftype])
            
#         if parent:
#             queryDict['parents'] = self.quote(parent)+' in parents'
                
        
#         if date:
#             queryDict['modifiedTime'] = 'modifiedTime >'+self.quote(date)
        
        
#         query = ' and '.join(queryDict.values())
#         print(query)
        
#         try:
#             result = self.service.files().list(q=query).execute()
#             return(result)
#         except gerror.HttpError as e:
#             print('File Not Found')
#             return(False)
        
# #         try:
# #             print('trying')
# #             result = self.service.files().list(q=query).execute()
# #         except gerror.HttpError as e:
# #             print('File Not Found')
# #             return(False)        
    
    
#     def search(self, name = None, trashed = False, ftype = False, fuzzy = False, parent = None):
        
        
#         if name is None:
#             print ('specify a folder or file name')
#             return(False)
#         else:
#             # this needs to be surrounded by double quotes
#             name = self.quote(name)
                
#         # give a reasonable value if this is null
#         if parent:
#             parentVal = self.quote(parent)            
#         else:
#             parentVal = self.quote('null')
            
#         if fuzzy:
#             fuzzyVal = ' contains '
#         else:
#             fuzzyVal = '='
            
#         supportedFeatures = {'name':'name'+fuzzyVal+self.quote(eval(name)), 
#                              'trashed': 'trashed='+eval(self.quote(trashed)), 
#                              'parent': self.quote(eval(parentVal))+' in parents',
#                              }
        
#         # exclude these if they are set to none or false
#         nullFeatures =['parent']
        
#         queryList = []
#         for each in supportedFeatures:
#             if each in nullFeatures and not eval(each):
#                 pass
#             else:
#                 queryList.append(str(supportedFeatures[each])) #+ str(myDict[each]))

#         query = ' and '.join(queryList)        
#         print(query)
        
#         try:
#             result = self.service.files().list(q=query).execute()
#         except gerror.HttpError as e:
#             print('File Not Found')
#             return(False)
        
#         return(result)

In [12]:
credentials = getCredentials()
# http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', credentials=credentials)


In [13]:
myDrive = gDrive(service)

In [17]:
# path to desktop of this user (this should work on macs)
desktopPath = pathify([os.path.expanduser('~')], 'Desktop')

# Fields expected in CSV file containing student data

Desktop/Users/aaronciuffo/


In [ ]:
service.files().get(fileId="0BzC-V2QIsGRGT00xSjJHNy1Td1k", fields='parents').execute()

In [ ]:
myDrive.parents('0BzC-V2QIsGRGT00xSjJHNy1Td1k')

In [14]:
myDrive.ls(name = 'schmoo', fuzzy = True, trashed = False)

name contains "schmoo" and trashed=False
name: release the schmoo!, ID:0BzC-V2QIsGRGT00xSjJHNy1Td1k, mimeType:application/vnd.google-apps.folder
name: release the schmoo!, ID:0BzC-V2QIsGRGbklHQUJ0ZzNDQmM, mimeType:application/vnd.google-apps.folder
name: schmooo!, ID:0BzC-V2QIsGRGWXNxNmhjc0FITDQ, mimeType:application/vnd.google-apps.folder


In [ ]:
response = myDrive.search(name = 'schmoo', fuzzy = True)
print (response)

In [ ]:
myDrive.addFile('the wifes sewing', ftype = 'folder', parents = "0BzC-V2QIsGRGWXNxNmhjc0FITDQ")

In [ ]:
myDrive.search(name = "more cows")

In [ ]:
discovery.mimetypes